In [30]:
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
import logging
import time
import sqlite3
import pandas as pd
from time import localtime
# import httperror
from urllib.error import HTTPError
import re

In [23]:

# USERNAME = 'coolpolyester'
# PASSWORD = 'Aaronjk12'

In [91]:
# Should not need to run this ever again. 
# cl = Client()
# cl.login(USERNAME, PASSWORD)
# cl.delay_range = [3, 10]
# cl.dump_settings("session.json")

True

In [4]:
logger = logging.getLogger()


def login_user(cl):
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """
    USERNAME = 'coolpolyester'
    PASSWORD = 'Aaronjk12'
    
    session = cl.load_settings("session.json")

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(USERNAME, PASSWORD)

            # check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                logger.info(
                    "Session is invalid, need to login via username and password"
                )

                old_session = cl.get_settings()

                # use the same device uuids across logins
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(USERNAME, PASSWORD)
            login_via_session = True
        except Exception as e:
            logger.info("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            logger.info(
                "Attempting to login via username and password. username: %s" % USERNAME
            )
            if cl.login(USERNAME, PASSWORD):
                login_via_pw = True
        except Exception as e:
            logger.info("Couldn't login user using username and password: %s" % e)

    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")

In [126]:
cl = Client()
login_user(cl)
# user_id = cl.user_id_from_username('actuarialclubuw')
# time.sleep(3)

In [124]:
conn = sqlite3.connect('clubs.db')
c = conn.cursor()
# instaUrls = c.execute(r'SELECT club_id, instagramUrl FROM clubs WHERE instagramUrl;').fetchall()

In [127]:
clubs_df = pd.read_sql_query("SELECT club_id, instagramUrl, insta_id FROM clubs;", conn)
# strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# cl.delay_range = [3, 10]

def extract_first_resource(url):
    # This regular expression will find the first resource after .com/ in the URL
    match = re.search(r'\.com/([^/?]+)', url)
    return match.group(1) if match else None


for index, club in clubs_df.iterrows():
    try:
        if club["instagramUrl"] != "" and club["insta_id"] is None:
            url = club["instagramUrl"]
            username = extract_first_resource(url)
            print(f'Requesting {username}\'s user_id')
            user_id = cl.user_id_from_username(username)

            # Put user_id into database
            c.execute('UPDATE clubs SET insta_id = ? WHERE club_id = ?;', (user_id, club['club_id']))
            conn.commit()

            formatted_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
            print(index, f'{club["instagramUrl"]}\'s user_id is {user_id} at {formatted_time}')
            
        elif club["instagramUrl"] != "":
            print(index, club["instagramUrl"] + "'s user_id is already in database")
    except HTTPError as e:
        print(e)
        continue

1 https://www.instagram.com/actuarialclubuw/'s user_id is already in database
2 https://www.instagram.com/alps_uw/'s user_id is already in database
3 https://www.instagram.com/adclubwisc's user_id is already in database
4 https://www.instagram.com/airwisco/'s user_id is already in database
5 https://www.instagram.com/ahana_maps/'s user_id is already in database
6 http://instagram.com/aigauw's user_id is already in database
7 https://www.instagram.com/ahswisconsin/'s user_id is already in database
9 http://www.instagram.com/alphachisigma_uw's user_id is already in database
10 https://www.instagram.com/alphadeltaphiwisco/'s user_id is already in database
11 https://www.instagram.com/aed.uwmadison/'s user_id is already in database
12 https://www.instagram.com/agr_uw/'s user_id is already in database
13 https://www.instagram.com/wisconsinakdphi/'s user_id is already in database
14 https://www.instagram.com/badgerakpsi's user_id is already in database
15 https://www.instagram.com/aomegae_uw

In [131]:
# Get media

user_ids = pd.read_sql_query("SELECT insta_id FROM clubs WHERE insta_id IS NOT NULL;", conn)
user_ids.iloc[0].values[0]

'23336856764'

In [132]:

media = cl.user_medias(user_ids.iloc[0].values[0], amount=10)
media

[Media(pk='3229701277872587834', id='3229701277872587834_23336856764', code='CzSNDNWOIQ6', taken_at=datetime.datetime(2023, 11, 6, 0, 26, 56, tzinfo=datetime.timezone.utc), media_type=8, image_versions2={}, product_type='', thumbnail_url=None, location=None, user=UserShort(pk='23336856764', username='actuarialclubuw', full_name='', profile_pic_url=None, profile_pic_url_hd=None, is_private=None, stories=[]), comment_count=1, comments_disabled=False, commenting_disabled_for_viewer=False, like_count=38, play_count=None, has_liked=None, caption_text='Halloween Social of 2023! Thank you to all who came out, it was a great turnout and a ton of fun. Turns out the actuaries know how to be creative! \n\nHappy Halloween! 🎃', accessibility_caption=None, usertags=[], sponsor_tags=[], video_url=None, view_count=0, video_duration=0.0, title='', resources=[Resource(pk='3229701272310920922', video_url=None, thumbnail_url=HttpUrl('https://scontent-msp1-1.cdninstagram.com/v/t51.2885-15/399537392_1146229

In [187]:
insta_ids = pd.read_sql_query("SELECT insta_id FROM clubs WHERE insta_id IS NOT NULL;", conn)
clubs = pd.read_sql_query("SELECT * FROM clubs;", conn)
seen_media = set(pd.read_sql_query("SELECT media_id FROM media;", conn)['media_id'])

for index, insta_id in insta_ids.iterrows():
    print(index)
    try:    
        media = cl.user_medias(insta_id, amount=10)
        for post in media:
            # Check if post is already in database
            if int(post.pk) in seen_media:
                pass
            else:
                # Insert post into database
                media_id = post.pk
                # club_id = clubs[clubs['insta_id'] == insta_id].iloc[0]['club_id']
                club_id = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_id'].values[0]
                club_name = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_name'].values[0]
                post_date = post.taken_at
                if post.resources:
                    first_image_url = post.resources[0].thumbnail_url
                else:
                    first_image_url = post.thumbnail_url
                media_src = 'Instagram'
                c.execute('INSERT INTO media VALUES (?, ?, ?, ?, ?, ?);', (media_id, club_id, first_image_url, media_src, post_date, club_name))
                seen_media.add(media_id)
                conn.commit()
                
    except Exception as e:
        print(e)
        continue


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
UNIQUE constraint failed: media.media_id
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
UNIQUE constraint failed: media.media_id
108
109
110
111
112
113
114
115
116
117
UNIQUE constraint failed: media.media_id
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246

In [197]:
insta_ids = pd.read_sql_query("SELECT insta_id FROM clubs WHERE insta_id IS NOT NULL;", conn)
clubs = pd.read_sql_query("SELECT * FROM clubs;", conn)
seen_media = set(pd.read_sql_query("SELECT media_id FROM media2;", conn)['media_id'])

for index, insta_id in insta_ids.iterrows():
    print(index)
    try:    
        media = cl.user_medias(insta_id, amount=10)
        for post in media:
            # Check if post is already in database
            if int(post.pk) in seen_media:
                pass
            else:
                # Insert post into database
                media_id = post.pk
                # club_id = clubs[clubs['insta_id'] == insta_id].iloc[0]['club_id']
                club_id = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_id'].values[0]
                club_name = clubs.loc[clubs['insta_id'] == insta_id.values[0]]['club_name'].values[0]
                post_date = post.taken_at
                if post.resources:
                    first_image_url = post.resources[0].thumbnail_url
                else:
                    first_image_url = post.thumbnail_url
                media_src = 'Instagram'
                media_caption = post.caption_text
                like_count = post.like_count
                c.execute('INSERT INTO media2 VALUES (?, ?, ?, ?, ?, ?, ?, ?);', (media_id, club_id, first_image_url, media_src, post_date, club_name, media_caption, like_count))
                seen_media.add(media_id)
                conn.commit()
                
    except Exception as e:
        print(e)
        continue

0
1


In [194]:
media[0].caption_text
media[0].like_count

39